In [2]:
# import libraries 
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns


In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

In [16]:
# import data 
df = pd.read_csv('spotify_songs.csv')

# check data
df.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [5]:
df.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000,18454.000000
mean,42.438821,0.644066,0.692289,5.342094,-6.769159,0.580525,0.106192,0.175348,0.051216,0.189593,0.520598,120.812167,230319.306763
std,24.616740,0.150071,0.180635,3.613121,2.920757,0.493487,0.102291,0.217795,0.168263,0.153751,0.228716,27.586424,57255.086685
min,0.000000,0.098500,0.016700,0.000000,-34.283000,0.000000,0.022400,0.000001,0.000000,0.009360,0.000010,37.114000,31893.000000
25%,24.000000,0.546000,0.573000,2.000000,-8.249000,0.000000,0.039700,0.016100,0.000000,0.092700,0.345000,98.856000,193230.250000
50%,48.000000,0.660000,0.715000,6.000000,-6.227000,1.000000,0.060300,0.081900,0.000009,0.128000,0.522000,120.045000,221340.000000
75%,62.000000,0.755000,0.835000,9.000000,-4.719000,1.000000,0.130000,0.254000,0.001720,0.246000,0.700000,135.984000,258078.250000
max,100.000000,0.979000,1.000000,11.000000,1.275000,1.000000,0.918000,0.992000,0.987000,0.996000,0.991000,214.047000,517810.000000


In [6]:
df.shape, df.columns, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18454 entries, 0 to 18453
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  18454 non-null  object 
 1   track_name                18454 non-null  object 
 2   track_artist              18454 non-null  object 
 3   lyrics                    18194 non-null  object 
 4   track_popularity          18454 non-null  int64  
 5   track_album_id            18454 non-null  object 
 6   track_album_name          18454 non-null  object 
 7   track_album_release_date  18454 non-null  object 
 8   playlist_name             18454 non-null  object 
 9   playlist_id               18454 non-null  object 
 10  playlist_genre            18454 non-null  object 
 11  playlist_subgenre         18454 non-null  object 
 12  danceability              18454 non-null  float64
 13  energy                    18454 non-null  float64
 14  key   

((18454, 25),
 Index(['track_id', 'track_name', 'track_artist', 'lyrics', 'track_popularity',
        'track_album_id', 'track_album_name', 'track_album_release_date',
        'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre',
        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'language'],
       dtype='object'),
 None)

In [15]:
# subset data for english lyrics only 
df = df[df['language'] == 'en']

In [14]:
top100 = df.nlargest(100, "track_popularity")
top100.sample(5)

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
14744,6DCZcSspjsKoFjzjrWoCdn,God's Plan,Drake,And they wishin' and wishin' and wishin' and w...,86,1ATL5GLyefJaxhQzSPVrLX,Scorpion,2018-06-29,ELECTROPOP🐹,44p8nNLe4fGfUeArS3MaIX,...,-9.211,1,0.1090,0.03320,0.000083,0.5520,0.357,77.169,198973,en
675,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,"Hahahahaha Tank God Ayy, ayy I've been fuckin'...",88,6trNtQUgC8cgbWcqoMYkOR,beerbongs & bentleys,2018-04-27,GTA San Andreas Radio Los Santos,6m0BqpJfSP2m5xCOFjDRHw,...,-6.136,0,0.0712,0.12400,0.000070,0.1310,0.129,159.801,218147,en
15631,6ocbgoVGwYJhOv1GgI9NsF,7 rings,Ariana Grande,"Yeah, breakfast at Tiffany's and bottles of bu...",90,2fYhqwDWXjbpjaIJPEfKFw,"thank u, next",2019-02-08,Intro to Post-Teen Pop,6o6MNYZqHSkMAKcCHPNu7K,...,-10.732,0,0.3340,0.59200,0.000000,0.0881,0.327,140.048,178627,en
17144,7CZyCXKG6d5ALeq41sLzbw,Take What You Want (feat. Ozzy Osbourne & Trav...,Post Malone,Oh I feel you crumble in my arms down to your ...,87,4g1ZRSobMefqF6nelkgibi,Hollywood's Bleeding,2019-09-06,Pop - Pop UK - 2019 - Canadian Pop - 2019 - Pop,46Cl6dmeiylK6TRGXr7hHe,...,-2.665,0,0.0502,0.09060,0.000000,0.1470,0.272,139.919,229573,en
12097,57vxBYXtHMk6H1aD29V7PU,Heartless,The Weeknd,"Young Metro, young Metro, young Metro (Sheesh)...",93,7vRcickwa6GCfY1qKKe4lK,Heartless,2019-11-27,Todo Éxitos,2ji5tRQVfnhaX1w9FhmSzk,...,-5.831,0,0.1110,0.00632,0.000076,0.1170,0.198,169.954,200080,en


In [12]:
# Preprocess text
top100['lyrics'] = top100['lyrics'].astype(str)
top100["lyrics"] = top100["lyrics"].str.lower().str.replace(r'[^\w\s]','').str.split()

# Fit tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
lyrics = top100["lyrics"].values.tolist()
tokenizer.fit_on_texts(lyrics)


C:\Users\htw10\AppData\Local\Temp\ipykernel_25644\241302411.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  top100["lyrics"] = top100["lyrics"].str.lower().str.replace(r'[^\w\s]','').str.split()


[['they',
  'say',
  'oh',
  'my',
  'god',
  'i',
  'see',
  'the',
  'way',
  'you',
  'shine',
  'take',
  'your',
  'hands',
  'my',
  'dear',
  'and',
  'place',
  'them',
  'both',
  'in',
  'mine',
  'you',
  'know',
  'you',
  'stopped',
  'me',
  'dead',
  'while',
  'i',
  'was',
  'passing',
  'by',
  'and',
  'now',
  'i',
  'beg',
  'to',
  'see',
  'you',
  'dance',
  'just',
  'one',
  'more',
  'time',
  'ooh',
  'i',
  'see',
  'you',
  'see',
  'you',
  'see',
  'you',
  'every',
  'time',
  'and',
  'oh',
  'my',
  'i',
  'i',
  'like',
  'your',
  'style',
  'you',
  'you',
  'make',
  'me',
  'make',
  'me',
  'make',
  'me',
  'wanna',
  'cry',
  'and',
  'now',
  'i',
  'beg',
  'to',
  'see',
  'you',
  'dance',
  'just',
  'one',
  'more',
  'time',
  'so',
  'they',
  'say',
  'dance',
  'for',
  'me',
  'dance',
  'for',
  'me',
  'dance',
  'for',
  'me',
  'ohohoh',
  'ive',
  'never',
  'seen',
  'anybody',
  'do',
  'the',
  'things',
  'you',
  'do',
  '

In [ ]:
# Define vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Create sequences from lyrics
sequences = tokenizer.texts_to_sequences(lyrics)
max_sequence_len = max([len(x) for x in sequences])

# Pad sequences
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding="pre")

# Create input and output sequences
sequences = tf.one_hot(sequences, depth=vocab_size)
sequences = tf.cast(sequences, tf.float32)

In [ ]:
# Train model
model = keras.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(None, vocab_size)))
model.add(keras.layers.Dense(vocab_size, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
# Train model
batch_size = 64
num_epochs = 10
steps_per_epoch = len(sequences) // batch_size
model.fit(sequences[:, :-1], sequences[:, 1:], epochs=num_epochs, batch_size=batch_size, steps_per_epoch=steps_per_epoch)